In [1]:
%pip install torch==1.9.1

Note: you may need to restart the kernel to use updated packages.


# Urinary system diseases classification
#### Presumptive classification of nephritis from inflammation

This notebook aims to create a classification model for the dataset described in here https://www.kaggle.com/yamqwe/acute-inflammationse


```
Credit:
Jacek Czerniak, Ph.D., Assistant ProfessorSystems Research InstitutePolish Academy of SciencesLaboratory of Intelligent Systems ul. Newelska 6, Room 21801-447 Warszawa, Polande-mail:
jacek.czerniak '@' ibspan.waw.pl or jczerniak '@' ukw.edu.pl
```

In [2]:
import pandas as pd
import sagemaker
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

## Feature engineering

### Load data

In [4]:
data_file = 'data/diagnosis.data'
columns = ['Temperature of patient', 'Occurrence of nausea', 'Lumbar pain', 'Urine pushing (continuous need for urination)', 'Micturition pains', 'Burning of urethra, itch, swelling of urethra outlet', 'Inflammation of urinary bladder', 'Nephritis of renal pelvis origin']
data_df = pd.read_csv(filepath_or_buffer=data_file, sep='\t', header=None, names=columns, encoding='utf-16')
data_df.head()

Temperature of patient Occurrence of nausea Lumbar pain  \
0                   35,5                   no         yes   
1                   35,9                   no          no   
2                   35,9                   no         yes   
3                   36,0                   no          no   
4                   36,0                   no         yes   

  Urine pushing (continuous need for urination) Micturition pains  \
0                                            no                no   
1                                           yes               yes   
2                                            no                no   
3                                           yes               yes   
4                                            no                no   

  Burning of urethra, itch, swelling of urethra outlet  \
0                                                 no     
1                                                yes     
2                                                 no     
3                                                yes     
4                                                 no     

  Inflammation of urinary bladder Nephritis of renal pelvis origin  
0                              no                               no  
1                             yes                               no  
2                              no                               no  
3                             yes                               no  
4                              no                               no

### Drop missing values

In [5]:
data_df.shape

(120, 8)

In [6]:
data_df = data_df.dropna(axis=0)
data_df.shape

(120, 8)

There are no missing values

### Convert text to numbers

In [7]:
for c in columns[1:]:
    data_df.loc[data_df[c]=='no',c] = 0
    data_df.loc[data_df[c]=='yes',c] = 1
data_df.head()

Temperature of patient Occurrence of nausea Lumbar pain  \
0                   35,5                    0           1   
1                   35,9                    0           0   
2                   35,9                    0           1   
3                   36,0                    0           0   
4                   36,0                    0           1   

  Urine pushing (continuous need for urination) Micturition pains  \
0                                             0                 0   
1                                             1                 1   
2                                             0                 0   
3                                             1                 1   
4                                             0                 0   

  Burning of urethra, itch, swelling of urethra outlet  \
0                                                  0     
1                                                  1     
2                                                  0     
3                                                  1     
4                                                  0     

  Inflammation of urinary bladder Nephritis of renal pelvis origin  
0                               0                                0  
1                               1                                0  
2                               0                                0  
3                               1                                0  
4                               0                                0

Convert temperature decimal character

In [8]:
data_df['Temperature of patient'] = data_df['Temperature of patient'].str.replace(',','.')
data_df.head()

Temperature of patient Occurrence of nausea Lumbar pain  \
0                   35.5                    0           1   
1                   35.9                    0           0   
2                   35.9                    0           1   
3                   36.0                    0           0   
4                   36.0                    0           1   

  Urine pushing (continuous need for urination) Micturition pains  \
0                                             0                 0   
1                                             1                 1   
2                                             0                 0   
3                                             1                 1   
4                                             0                 0   

  Burning of urethra, itch, swelling of urethra outlet  \
0                                                  0     
1                                                  1     
2                                                  0     
3                                                  1     
4                                                  0     

  Inflammation of urinary bladder Nephritis of renal pelvis origin  
0                               0                                0  
1                               1                                0  
2                               0                                0  
3                               1                                0  
4                               0                                0

### Create labels  

- Check if one patient can have both diseases

In [9]:
data_df[(data_df['Inflammation of urinary bladder'] == 1) & (data_df['Nephritis of renal pelvis origin'] == 1)].shape

(19, 8)

- Check if one patient don't have any disease

In [10]:
data_df[(data_df['Inflammation of urinary bladder'] == 0) & (data_df['Nephritis of renal pelvis origin'] == 0)].shape

(30, 8)

Create a Diagnosis column which will serve as our label. It can have values of
- 0 : if patient has no disease
- 1 : if patient was diagnosed with just "Nephritis of renal pelvis origin"
- 2 : if patient was diagnosed with just "Inflammation of urinary bladder"
- 3 : if patient was diagnosed with both diseases

In [11]:
conditions = [
    ((data_df['Inflammation of urinary bladder'] ==
     0) & (data_df['Nephritis of renal pelvis origin'] == 0)),
    ((data_df['Inflammation of urinary bladder'] ==
     0) & (data_df['Nephritis of renal pelvis origin'] == 1)),
    ((data_df['Inflammation of urinary bladder'] ==
     1) & (data_df['Nephritis of renal pelvis origin'] == 0)),
    ((data_df['Inflammation of urinary bladder'] ==
     1) & (data_df['Nephritis of renal pelvis origin'] == 1))
]
values = [0, 1, 2, 3]
data_df['Diagnosis'] = np.select(conditions, values)
data_df.head()


Temperature of patient Occurrence of nausea Lumbar pain  \
0                   35.5                    0           1   
1                   35.9                    0           0   
2                   35.9                    0           1   
3                   36.0                    0           0   
4                   36.0                    0           1   

  Urine pushing (continuous need for urination) Micturition pains  \
0                                             0                 0   
1                                             1                 1   
2                                             0                 0   
3                                             1                 1   
4                                             0                 0   

  Burning of urethra, itch, swelling of urethra outlet  \
0                                                  0     
1                                                  1     
2                                                  0     
3                                                  1     
4                                                  0     

  Inflammation of urinary bladder Nephritis of renal pelvis origin  Diagnosis  
0                               0                                0          0  
1                               1                                0          2  
2                               0                                0          0  
3                               1                                0          2  
4                               0                                0          0

In [12]:
data_df = data_df.astype({'Temperature of patient':'float',
        'Occurrence of nausea':'float',
        'Lumbar pain':'float',
        'Urine pushing (continuous need for urination)':'float',
        'Micturition pains':'float',
        'Burning of urethra, itch, swelling of urethra outlet':'float',
        'Inflammation of urinary bladder':'int32',
        'Nephritis of renal pelvis origin':'int32',
        'Diagnosis':'int32',
})
data_df.dtypes

Temperature of patient                                  float64
Occurrence of nausea                                    float64
Lumbar pain                                             float64
Urine pushing (continuous need for urination)           float64
Micturition pains                                       float64
Burning of urethra, itch, swelling of urethra outlet    float64
Inflammation of urinary bladder                           int32
Nephritis of renal pelvis origin                          int32
Diagnosis                                                 int32
dtype: object

In [13]:
data_df: pd.DataFrame
data_df.describe(include='all')

Temperature of patient  Occurrence of nausea  Lumbar pain  \
count              120.000000            120.000000   120.000000   
mean                38.724167              0.241667     0.583333   
std                  1.819132              0.429888     0.495074   
min                 35.500000              0.000000     0.000000   
25%                 37.100000              0.000000     0.000000   
50%                 37.950000              0.000000     1.000000   
75%                 40.600000              0.000000     1.000000   
max                 41.500000              1.000000     1.000000   

       Urine pushing (continuous need for urination)  Micturition pains  \
count                                     120.000000         120.000000   
mean                                        0.666667           0.491667   
std                                         0.473381           0.502027   
min                                         0.000000           0.000000   
25%                                         0.000000           0.000000   
50%                                         1.000000           0.000000   
75%                                         1.000000           1.000000   
max                                         1.000000           1.000000   

       Burning of urethra, itch, swelling of urethra outlet  \
count                                         120.000000      
mean                                            0.416667      
std                                             0.495074      
min                                             0.000000      
25%                                             0.000000      
50%                                             0.000000      
75%                                             1.000000      
max                                             1.000000      

       Inflammation of urinary bladder  Nephritis of renal pelvis origin  \
count                       120.000000                        120.000000   
mean                          0.491667                          0.416667   
std                           0.502027                          0.495074   
min                           0.000000                          0.000000   
25%                           0.000000                          0.000000   
50%                           0.000000                          0.000000   
75%                           1.000000                          1.000000   
max                           1.000000                          1.000000   

        Diagnosis  
count  120.000000  
mean     1.400000  
std      1.032253  
min      0.000000  
25%      0.750000  
50%      1.000000  
75%      2.000000  
max      3.000000

### Data exploration

In [14]:
fig, ax =plt.subplots(len(data_df.columns),1, figsize=(20,4*len(data_df.columns)))
for i, c in enumerate(data_df.columns):
    sns.countplot(data=data_df, x=c, ax=ax[i])
fig.show()

There are more patients with temperatures less than 38 celcius (low fever), but the number of patients with more than 40 celcius (high fever) is almost the same. But we can see that there are distinct groups.
The most imbalanced input is Occurency of Nausea.

### Scale temperature

In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
data_df[['Temperature of patient']]=pd.DataFrame(scaler.fit_transform(data_df[['Temperature of patient']].astype(float)))
data_df.head()

Temperature of patient  Occurrence of nausea  Lumbar pain  \
0                0.000000                   0.0          1.0   
1                0.066667                   0.0          0.0   
2                0.066667                   0.0          1.0   
3                0.083333                   0.0          0.0   
4                0.083333                   0.0          1.0   

   Urine pushing (continuous need for urination)  Micturition pains  \
0                                            0.0                0.0   
1                                            1.0                1.0   
2                                            0.0                0.0   
3                                            1.0                1.0   
4                                            0.0                0.0   

   Burning of urethra, itch, swelling of urethra outlet  \
0                                                0.0      
1                                                1.0      
2                                                0.0      
3                                                1.0      
4                                                0.0      

   Inflammation of urinary bladder  Nephritis of renal pelvis origin  \
0                                0                                 0   
1                                1                                 0   
2                                0                                 0   
3                                1                                 0   
4                                0                                 0   

   Diagnosis  
0          0  
1          2  
2          0  
3          2  
4          0

In [83]:
scaler.data_max_, scaler.data_min_, scaler.data_range_

(array([41.5]), array([35.5]), array([6.]))

### Correlation matrix

In [16]:
corr_matrix = data_df.corr().abs().round(2)
sns.heatmap(corr_matrix, annot=True, cmap='bwr', linewidths=.5)

<AxesSubplot:>

Inflamation is correlated to Urine pushing and Micturition pains, while Nephritis is correlated to Temperature, Nausea and Lumbar pain. 

### Drop previous diagnosis columns

In [17]:
data_df = data_df.drop(['Inflammation of urinary bladder', 'Nephritis of renal pelvis origin'], axis=1)
data_df.head()

Temperature of patient  Occurrence of nausea  Lumbar pain  \
0                0.000000                   0.0          1.0   
1                0.066667                   0.0          0.0   
2                0.066667                   0.0          1.0   
3                0.083333                   0.0          0.0   
4                0.083333                   0.0          1.0   

   Urine pushing (continuous need for urination)  Micturition pains  \
0                                            0.0                0.0   
1                                            1.0                1.0   
2                                            0.0                0.0   
3                                            1.0                1.0   
4                                            0.0                0.0   

   Burning of urethra, itch, swelling of urethra outlet  Diagnosis  
0                                                0.0             0  
1                                                1.0             2  
2                                                0.0             0  
3                                                1.0             2  
4                                                0.0             0

### Split train and test data

In [27]:
test_ratio = 0.2
test_index = int(data_df.shape[0]*(1-test_ratio))
train_df = data_df[:test_index]
test_df = data_df[test_index:]
train_X, train_Y = train_df[train_df.columns[:-1]], train_df[train_df.columns[-1]]
test_X, test_Y = test_df[test_df.columns[:-1]], test_df[test_df.columns[-1]]

### Save locally

In [28]:
data_dir = "urine-data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
pd.concat([train_Y, train_X], axis=1).to_csv(os.path.join(data_dir, "train.csv"), header=False, index=False)
pd.concat([test_Y, test_X], axis=1).to_csv(os.path.join(data_dir, "test.csv"), header=False, index=False)

### Upload to S3

In [29]:
prefix = 'project-urine'

train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)
test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)

### Create AWS model

In [36]:
from sagemaker.pytorch import PyTorch

INPUT_DIM = train_X.shape[1]
HIDDEN_DIM = train_X.shape[1]*8
OUTPUT_DIM = 4

output_path = 's3://{}/{}'.format(bucket, prefix)
estimator = PyTorch(entry_point='train.py',
                    source_dir='source',
                    role=role,
                    framework_version='1.9.1',
                    py_version='py38',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_features': INPUT_DIM,
                        'hidden_dim': HIDDEN_DIM,
                        'output_dim': OUTPUT_DIM,
                        'epochs': 300
                    })

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Train model in AWS

In [37]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': train_location})

2022-02-22 19:20:19 Starting - Starting the training job...
2022-02-22 19:20:41 Starting - Launching requested ML instancesProfilerReport-1645557619: InProgress
.........
2022-02-22 19:22:11 Starting - Preparing the instances for training.........
2022-02-22 19:23:43 Downloading - Downloading input data...
2022-02-22 19:24:03 Training - Downloading the training image...
2022-02-22 19:24:50 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-02-22 19:24:51,485 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-02-22 19:24:51,486 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-02-22 19:24:51,495 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-02-22 19:24:51,504 sagemaker_pytorch_container.training INFO     Invoking user training

In [38]:
estimator.model_data

's3://sagemaker-us-east-1-801008216402/project-urine/pytorch-training-2022-02-22-19-20-19-115/output/model.tar.gz'

### Deploy trained model

In [ ]:
from sagemaker.pytorch import PyTorchModel

In [45]:
%%time

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.9.1',
                     py_version='py38',
                     entry_point='predict.py',
                     source_dir='source')

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


---------------!CPU times: user 444 ms, sys: 20.8 ms, total: 465 ms
Wall time: 7min 32s


In [67]:
%%time

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.9.1',
                     py_version='py38',
                     entry_point='predict.py',
                     source_dir='source')

# deploy your model to create a predictor
predictor_csv = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-----------!CPU times: user 536 ms, sys: 8.62 ms, total: 544 ms
Wall time: 5min 32s


### Evaluate

In [48]:
output = predictor.predict(test_X)

80

In [53]:
test_y_preds = output.argmax(axis=1)
test_y_preds

array([1, 1, 3, 3, 1, 3, 0, 1, 1, 3, 3, 0, 1, 1, 3, 0, 1, 1, 3, 1, 0, 1,
       1, 1])

In [52]:
test_Y.values

array([1, 1, 3, 3, 1, 3, 0, 1, 1, 3, 3, 0, 1, 1, 3, 0, 1, 1, 3, 1, 0, 1,
       1, 1], dtype=int32)

In [55]:
accuracy = np.logical_and(test_Y.values, test_y_preds).sum()/len(test_Y)
accuracy

0.8333333333333334

In [77]:
output_train = predictor.predict(train_X)

In [78]:
train_y_preds = output_train.argmax(axis=1)
train_y_preds

array([0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2,
       0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 3, 3, 0, 0, 1, 1, 1, 3, 3, 0, 1, 1, 3, 3, 3, 0, 1,
       3, 3, 0, 1, 3, 3, 0, 1])

In [79]:
accuracy_train = np.logical_and(train_Y.values, train_y_preds).sum()/len(train_Y)
accuracy_train

0.7291666666666666

In [72]:
none = data_df[data_df['Diagnosis']==0].copy()
none['Temperature of patient'] = none['Temperature of patient']*(41.5-35.5)+35.5
none.head()

Temperature of patient  Occurrence of nausea  Lumbar pain  \
0                    35.5                   0.0          1.0   
2                    35.9                   0.0          1.0   
4                    36.0                   0.0          1.0   
5                    36.0                   0.0          1.0   
7                    36.2                   0.0          1.0   

   Urine pushing (continuous need for urination)  Micturition pains  \
0                                            0.0                0.0   
2                                            0.0                0.0   
4                                            0.0                0.0   
5                                            0.0                0.0   
7                                            0.0                0.0   

   Burning of urethra, itch, swelling of urethra outlet  Diagnosis  
0                                                0.0             0  
2                                                0.0             0  
4                                                0.0             0  
5                                                0.0             0  
7                                                0.0             0

In [74]:
nephritis = data_df[data_df['Diagnosis']==1].copy()
nephritis['Temperature of patient'] = nephritis['Temperature of patient']*(41.5-35.5)+35.5
nephritis.head()

Temperature of patient  Occurrence of nausea  Lumbar pain  \
60                    38.0                   0.0          1.0   
61                    38.0                   0.0          1.0   
62                    38.1                   0.0          1.0   
63                    38.3                   0.0          1.0   
64                    38.5                   0.0          1.0   

    Urine pushing (continuous need for urination)  Micturition pains  \
60                                            1.0                0.0   
61                                            1.0                0.0   
62                                            1.0                0.0   
63                                            1.0                0.0   
64                                            1.0                0.0   

    Burning of urethra, itch, swelling of urethra outlet  Diagnosis  
60                                                1.0             1  
61                                                1.0             1  
62                                                1.0             1  
63                                                1.0             1  
64                                                1.0             1

In [75]:
inflammation = data_df[data_df['Diagnosis']==2].copy()
inflammation['Temperature of patient'] = inflammation['Temperature of patient']*(41.5-35.5)+35.5
inflammation.head()

Temperature of patient  Occurrence of nausea  Lumbar pain  \
1                    35.9                   0.0          0.0   
3                    36.0                   0.0          0.0   
6                    36.2                   0.0          0.0   
8                    36.3                   0.0          0.0   
9                    36.6                   0.0          0.0   

   Urine pushing (continuous need for urination)  Micturition pains  \
1                                            1.0                1.0   
3                                            1.0                1.0   
6                                            1.0                1.0   
8                                            1.0                1.0   
9                                            1.0                1.0   

   Burning of urethra, itch, swelling of urethra outlet  Diagnosis  
1                                                1.0             2  
3                                                1.0             2  
6                                                1.0             2  
8                                                1.0             2  
9                                                1.0             2

In [76]:
both = data_df[data_df['Diagnosis']==3].copy()
both['Temperature of patient'] = both['Temperature of patient']*(41.5-35.5)+35.5
both.head()

Temperature of patient  Occurrence of nausea  Lumbar pain  \
70                    40.0                   1.0          1.0   
71                    40.0                   1.0          1.0   
72                    40.0                   1.0          1.0   
78                    40.1                   1.0          1.0   
79                    40.2                   1.0          1.0   

    Urine pushing (continuous need for urination)  Micturition pains  \
70                                            1.0                1.0   
71                                            1.0                1.0   
72                                            1.0                1.0   
78                                            1.0                1.0   
79                                            1.0                1.0   

    Burning of urethra, itch, swelling of urethra outlet  Diagnosis  
70                                                1.0             3  
71                                                1.0             3  
72                                                0.0             3  
78                                                0.0             3  
79                                                1.0             3